# MVP Modeling

##### mariadb pass

## intro

### setup

#### imports

###### basic

In [23]:
import pandas as pd, numpy as np, os, re, pyperclip, pickle
import mysql.connector as mysql, sqlalchemy
from datetime import datetime as dt, timedelta
import datetime

###### fonts

https://github.com/linzino7/matplotlibChinesefix

In [2]:
from matplotlib.font_manager import findfont, FontProperties  
findfont(FontProperties(family=FontProperties().get_family())) 

'/home/bhrdwj/anaconda3/envs/py39/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'

In [3]:
import matplotlib 
matplotlib.matplotlib_fname()

'/home/bhrdwj/anaconda3/envs/py39/lib/python3.9/site-packages/matplotlib/mpl-data/matplotlibrc'

In [4]:
import matplotlib.pyplot as plt 
plt.rcParams['font.sans-serif'] = ['Noto Sans CJK JP']  
plt.rcParams['axes.unicode_minus'] = False 

#### connect to mariadb

##### connect

##### test

#### some paths

In [5]:
rawdumps_path = '../data/raw/jawiki/dumps_unzipped/'
processed_path = '../data/processed/jawiki/'
twitter_path = '../data/raw/twitter/'

#### END

### functions  
<mark>unfinished:</mark>  google translate

#### simple functions to read from MySQL dumpfiles

##### function ```jpr_sql``` 

In [17]:
def jpr_sql(query, conn=conn):
    """
    Super-basic wrapper of SQLAlchemy conn.execute()
    Input:
        query string
        conn SQLAlchemy connection
    Output:
        list / result of conn.execute().fetchall()
    """
    try: 
        a = conn.execute(query)
    except Exception as e:
        print('Printing execute error:')
        print(e)
        return None
    
    try:
        return a.fetchall()
    except Exception as e:
        print('Printing results error:')
        print(e)
        print('Printing result')
        return None

##### function ```byte_read_sql```

#### google translate / add to cache <mark> UNFINISHED </mark>

#### functions to process MySQL dumpfiles

##### in cross-section

###### get_all_descendants_by_namespace

###### create_table_where_pageid_in_list

###### get_all_descendants_content_pageids

##### in time series

###### function to count all edits to pages in a whole timeseries table

###### query to **explore** revisions within subcats by japanese day

###### query to **count** revisions within subcats by japanese day

In [6]:
def count_revs_in_subcat_daily(subcat_name, subcat_descendant_ids:list[int], table:str) -> pd.DataFrame:
    """
    Count revision-edits of content-descendants of subcat, by day.
    """
    
    sql = """
    select
        date(date_tmstp_jp) as day
        ,count(row_id) as """+ subcat_name +r"""
    from 
        """+ table +r"""
    where
        page_id in ("""+ ", ".join(map(str,subcat_descendant_ids)) +r""")
        and event_entity = 'revision'
        and user_is_bot_by IS NULL
        and event_user_is_bot_by IS NULL
    group by day
    order by day
    ;
    """
    return pd.read_sql(sql,conn) #.set_index('day')

#### function to be my simple API: ```revcounts_within_page_id```

##### function takes page_id of a subcat or page, and returns df of daily edit counts

### unpickle

#### load pickled ```disaster_descendants```

In [7]:
with open(processed_path + 'disaster_descendants_raw.pickle', 'rb') as f:
    disaster_descendants_raw = pickle.load(f)

#### load pickled ```subcat_content_pageids```

In [8]:
with open(processed_path + 'subcat_content_pageids.pickle', 'rb') as f:
    subcat_content_pageids = pickle.load(f)

#### load pickled ```subcat_edit_counts```

In [9]:
with open(processed_path + 'subcat_edit_counts.pickle', 'rb') as f:
    subcat_edit_counts = pickle.load(f)

#### load pickled ```bousai``` (official disaster tweets)

In [10]:
with open(twitter_path + 'jma_bousai.pickle', 'rb') as f:
    bousai = pickle.load(f)

## get time series df

### build calendar

#### get calendar boundaries

In [25]:
first_twitter_date = bousai.index.min()
first_twitter_date = dt.date(first_twitter_date)

# last_target_date = jpr_sql('select max(date_tmstp_jp) from h2021;')[0][0]
# last_target_date = dt.date(last_target_date)
last_target_date = datetime.date(2022, 1, 1)

In [ ]:
first_twitter_date, last_target_date

In [ ]:
first_date = first_twitter_date + timedelta(days=1)
last_date = last_target_date - timedelta(days=1)

#### make blank calendar

In [ ]:
dates = pd.date_range(start=first_date, end=last_date, freq='D', tz='Japan')  # .map(lambda x: dt.date(x))

### prep and viz twitter features

##### get and viz twitter features

In [ ]:
bousai_days = bousai.iloc[:, 6:].resample('D').sum().fillna(0)
bousai_days = bousai_days.loc[dates]

#### prep ```mtwtf```, ```satsun```, ```holiday_on_weekday```

In [ ]:
bousai_days['mtwtf'] = bousai_days.index.dayofweek.isin([0,1,2,3,4]).astype(int)
bousai_days['satsun'] = bousai_days.index.dayofweek.isin([5,6]).astype(int)
# bousai_days['holiday_on_weekday'] = bousai_days.index.dayofweek.isin([5,6])

In [ ]:
from holidays_jp import CountryHolidays   # a little skeptical of these holidays -- will check them later
# get japanese holidays in 2015.
holidays2019 = CountryHolidays.get('JP', 2019)
holidays2020 = CountryHolidays.get('JP', 2020)
holidays2021 = CountryHolidays.get('JP', 2021)
holiday_list = [holidays2019, holidays2020, holidays2021]
holidays = pd.concat(map(pd.DataFrame, holiday_list), axis='rows').set_index(0)

In [ ]:
holidays.index = holidays.index.tz_localize('Japan')
holidays = holidays.resample('D').asfreq(' ').rename(columns={1:'holiday_name'})
holidays['holiday'] = holidays.holiday_name.map(lambda x: int(x != ' '))

In [ ]:
bousai_days = bousai_days.join(holidays, how='left')
bousai_days['holiday_name'] = bousai_days.holiday_name.fillna(' ')
bousai_days['holiday'] = bousai_days.holiday.fillna(0).astype(int)
bousai_days['holiday_on_weekday'] = bousai_days[['holiday', 'mtwtf']].all(axis='columns').astype(int)

### prep targets

#### clean disaster_descendants

##### some globals

In [ ]:
disaster_cat_page_ids = {'火山災害':2390743, '熱帯低気圧':626482, '雪害':2390774, '地震':135264, '津波':765772}  # '自然災害':137069, 
disasters_english = {'火山災害':'VolcanicDisaster', '熱帯低気圧':'TropicalCyclones', '雪害':'SnowDamage', '地震':'Earthquake', '津波':'Tsunami'}
disasters = list(disaster_cat_page_ids.keys())

##### drop rows with duplicate category name for the same category page_id

In [ ]:
disaster_descendants = {}
for i in disaster_descendants_raw:
    disaster_descendants[i] = disaster_descendants_raw[i].drop_duplicates(subset='id')
    disaster_descendants[i] = disaster_descendants[i][disaster_descendants[i].namespace == 0]

#### get disaster_daily_edits

In [ ]:
with open(processed_path + 'disaster_daily_edits.pickle', 'rb') as f:
    disaster_daily_edits = pickle.load(f)

#### combine & resample targets with features for train-test-split

In [ ]:
df = bousai_days.join(disaster_daily_edits, how='left')

## EDA

#### means and std

In [ ]:
df.describe().T[['std','mean']]

In [ ]:
df.describe().T.apply(lambda x: x['std']/x['mean'], axis=1)  #df.shift(-1)

#### plot twitter features

In [ ]:
df.iloc[:,:13].plot(figsize=(20,5), cmap='viridis')
plt.show()

#### plot targets

In [ ]:
df[['火山災害','熱帯低気圧','雪害','地震','津波']].plot(figsize=(20,5), cmap='viridis')
plt.show()

## Prep for regression

In [ ]:
dzs = ['火', '雨', '震', '雪', '津']  # disasters abbreviated
target_names = ['火山災害','熱帯低気圧','地震','雪害','津波']
calendar_vars = ['mtwtf','satsun','holiday_on_weekday']
twitter_feature_names = (
    ['火_火山',      '火_噴火警戒レベル1','火_火口周辺規制','火_入山規制', 
     '雨_警戒レベル5','雨_大雨',          '雨_土砂災害',
     '震_地震',      '震_震度5',         '震_震度678',
     '雪_大雪',      '雪_路面凍結',
     '津_津波']
)

In [ ]:
def get_onset_series(ser):
    return (
        pd.DataFrame((
            ser.shift(1)>0, 
            ser.shift(2)==0
        )).T.dropna()
        .all(axis=1)
        .astype(int)
    )

In [ ]:
df_lags = df.copy()
tw_l1_vars = [i + '_l1' for i in twitter_feature_names]
tw_ons_vars =   [i + '_ons' for i in twitter_feature_names] # onset
tg_l1_vars = [i + '_l1' for i in target_names]
tg_μ7_vars =    [i + '_μ7' for i in target_names] # rolling 7-day mean
for i in twitter_feature_names + target_names:
    df_lags[i+'_l1'] = df_lags[i].shift(1)
for i in twitter_feature_names:
    df_lags[i+'_ons'] = get_onset_series(df_lags[i])
for i in target_names:
    df_lags[i+'_μ7'] = df_lags[i].rolling(window=7, closed="left").mean()
df_lags = df_lags.dropna()

##### EDA on the rolling average variable

In [ ]:
df_lags[df_lags.columns[df_lags.columns.str.startswith('熱帯低気圧')]].iloc[280:300]

##### EDA on the onset variable

In [ ]:
df_lags[df_lags.columns[df_lags.columns.str.startswith('雨_大雨')]].iloc[280:300]

#### prep

In [ ]:
YY = df_lags[target_names]

In [ ]:
XX = df_lags[tw_l1_vars + tw_ons_vars + tg_l1_vars + tg_μ7_vars + calendar_vars]

In [ ]:
end_tr = 400
end_vl = 500
XXtr, YYtr = XX.copy().iloc[:end_tr], YY.iloc[:end_tr]
XXte, YYte = XX.copy().iloc[end_tr:end_vl], YY.iloc[end_tr:end_vl]

In [ ]:
dzs

In [ ]:
target_names

In [ ]:
Xcols = {}
for i,j in zip(dzs, target_names):
    Xcols[i] = (XX.columns[XX.columns.str.startswith(i)].tolist() + 
                   XX.columns[XX.columns.str.startswith(j)].tolist() +
                   calendar_vars
                 )

# Xcols['火'] = XX.columns[XX.columns.str.startswith('火')].tolist() + XX.columns.str.startswith('火山災害').tolist()+calendar_vars
# Xcols['雨'] = XX.columns.str.startswith('雨').tolist() + XX.columns.str.startswith('熱帯低気圧').tolist()+calendar_vars
# Xcols['震'] = XX.columns.str.startswith('震').tolist() + XX.columns.str.startswith('地震').tolist()+calendar_vars
# Xcols['雪'] = XX.columns.str.startswith('雪').tolist() + XX.columns.str.startswith('雪害').tolist()+calendar_vars
# Xcols['津'] = XX.columns.str.startswith('津').tolist() + XX.columns.str.startswith('津波').tolist()+calendar_vars
# # Xcols['時'] = ['mtwtf','satsun','holiday_on_weekday']

ycols = {'火':'火山災害','雨':'熱帯低気圧','震':'地震','雪':'雪害','津':'津波','時':'地震'}

In [ ]:
Xtr, Xte = {}, {}
ytr, yte = {}, {}
for diz in Xcols:
    Xtr[diz] = XXtr[Xcols[diz]]
    Xte[diz] = XXte[Xcols[diz]]
    ytr[diz] = YYtr[ycols[diz]]
    yte[diz] = YYte[ycols[diz]]

## Try Models

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
ytr.keys(), Xtr.keys()

In [ ]:
lr = {}
for diz in Xtr:
    lr[diz] = LinearRegression()
    lr[diz].fit(Xtr[diz],ytr[diz])
    print(lr[diz].score(Xte[diz],yte[diz]))

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
lr = {}
for diz in Xtr:
    lr[diz] = RandomForestRegressor(max_depth=4)
    lr[diz].fit(Xtr[diz],ytr[diz])
    print(lr[diz].score(Xte[diz],yte[diz]))

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
lr = {}
for diz in dizs:
    lr[diz] = RandomForestRegressor(max_depth=1)
    lr[diz].fit(Xtr[diz],ytr[diz])
    print(lr[diz].score(Xte[diz],yte[diz]))